In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("/content/drive/MyDrive/HomeC.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0


In [4]:
data['time'] = pd.DatetimeIndex(pd.date_range('2016-01-01 05:00', periods=len(data),  freq='min'))
pd.to_datetime(data['time'], unit='s');

In [5]:
data.columns

Index(['time', 'use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
       'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]',
       'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]',
       'Kitchen 14 [kW]', 'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]',
       'Microwave [kW]', 'Living room [kW]', 'Solar [kW]', 'temperature',
       'icon', 'humidity', 'visibility', 'summary', 'apparentTemperature',
       'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity',
       'dewPoint', 'precipProbability'],
      dtype='object')

# month separation

In [6]:
data['month'] = pd.DatetimeIndex(data['time']).month
data['Day'] = pd.DatetimeIndex(data['time']).day
data.dropna(inplace=True)

In [7]:
train=data

In [11]:

l=[]
l2=[]
for j in range (1,13):
  train = data.loc[data['month']==j]
  #print(train)

  # #print(train['Day'].max())
  for i in range(1,(train['Day'].max())):
    #print(j)
    summ=train[train['Day']==i].loc[:,['use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
       'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]',
       'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]',
       'Kitchen 14 [kW]', 'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]',
       'Microwave [kW]', 'Living room [kW]']].sum()
    summ.to_frame().reset_index()
    summ['month']=j
    summ['day']=i
    
    l.append(summ)
    averages=train[train['Day']==i].loc[:,['temperature','humidity']].mean()
    averages.to_frame().reset_index()
    l2.append(averages)

  



    
    #print()
    #print(averages)

In [12]:
vertical_concat = pd.concat((l[i]  for i in range(len(l))), axis=1)

In [13]:
average = pd.concat((l2[i]  for i in range(len(l2))), axis=1)

In [14]:
average.head()

,0,1,2,3,4,5,6,7,8,9,...,329,330,331,332,333,334,335,336,337,338
temperature,35.725737,31.692569,33.273625,22.297722,11.950139,19.975833,23.332667,29.488389,37.799556,45.367097,...,19.968528,28.520021,34.485486,34.945208,38.084437,26.570861,43.898000,35.555826,29.963292,32.682153
humidity,0.631825,0.580056,0.613597,0.598597,0.535292,0.572986,0.622042,0.717472,0.797667,0.842292,...,0.613424,0.727167,0.694896,0.690097,0.753056,0.540438,0.792965,0.617097,0.782819,0.753951


In [15]:
average.index

Index(['temperature', 'humidity'], dtype='object')

In [16]:
df1=average.T
df2=vertical_concat.T

In [17]:
omnia=df1.join(df2)

target=omnia['use [kW]']
features=omnia.drop(['use [kW]','gen [kW]', 'House overall [kW]',
       'Furnace 1 [kW]', 'Home office [kW]',
       'Wine cellar [kW]', 'Kitchen 12 [kW]', 'Kitchen 38 [kW]', 'Well [kW]',
       'Living room [kW]','day'],axis = 1)

features.head()

,temperature,humidity,Dishwasher [kW],Furnace 2 [kW],Fridge [kW],Garage door [kW],Kitchen 14 [kW],Barn [kW],Microwave [kW],month
0,35.725737,0.631825,59.517171,260.196833,94.740300,15.228367,10.935817,99.622950,12.961117,1.0
1,31.692569,0.580056,63.154404,359.091400,74.851733,19.246333,14.503251,185.776967,12.480300,1.0
2,33.273625,0.613597,105.508668,338.935050,92.149033,19.444933,18.030351,146.493667,19.066117,1.0
3,22.297722,0.598597,58.880760,318.407883,71.016683,19.249983,16.210219,47.837017,20.249933,1.0
4,11.950139,0.535292,72.973254,387.997850,80.135633,19.773067,13.866869,50.166067,16.658967,1.0


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,target,
                                                    test_size=0.2)

In [26]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
y_pred= reg.predict(X_train)

In [27]:
import sklearn
r2 = sklearn.metrics.r2_score(y_train, y_pred)


In [28]:
r2

0.6926611517692086

In [70]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.3)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))

Ridge model: [ 14.58120455 -21.50751124   1.19857492   5.98951899  10.11717645
   3.61744755  11.26157785   1.09670314   6.7145482   35.30432288]


In [71]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.6926610582681795
0.597443331314714


In [72]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.2)
lasso.fit(X_train,y_train)
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))

0.6926547063523001
0.5972338254682883
